# Name entity recognition on the IPU

Integration of the Graphcore Intelligence Processing Unit (IPU) and the Hugging Face transformer library means that it only takes a few lines of code to perform complex tasks which require deep learning.

In this notebook we perform **name entity extraction (NER)**  also known as token classification: we use natural language processing models to classify the words inside the prompt. 


The ease of use of the `pipeline` interface lets us quickly experiment with the pre-trained models and identify which one will work best.
This simple interface means that it is extremely easy to access the fast inference performance of the IPU on your application.

<img src="images/token_classification.png" alt="Widget inference on a token classification task" style="width:500px;">

While this notebook is focused on using the model (inference), our [token_classification](token_classification.ipynb) notebook will show you how to fine tune a model for a specific task using the [`datasets`](https://huggingface.co/docs/datasets/index) package.

In order to run this demo you will need to have a Poplar SDK environment enabled with the PopTorch installed
(see the [Getting Started](https://docs.graphcore.ai/en/latest/getting-started.html) guide for your IPU system),  and Optimum Graphcore.

First of all, let's make sure your environment has the latest version of [🤗 Optimum Graphcore](https://github.com/huggingface/optimum-graphcore) available.

In [ ]:
%pip install "optimum-graphcore>=0.4, <0.5"
%pip install emoji==0.6.0 gradio

In [ ]:
%load_ext autoreload
%autoreload 2

The value for cache directories can be configured through environment variables or directly in the notebook:

In [ ]:
import os
executable_cache_dir = os.getenv("POPLAR_EXECUTABLE_CACHE_DIR", "./exe_cache/")

## NER with transformers pipelines on the IPU

The simplest way to use a model on the IPU is to use the `pipeline` function. It provides a set of models which have been validated to work on a given task. To get started choose the task and call the `pipeline` function:

In [ ]:
from optimum.graphcore import pipelines
inference_config = dict(layers_per_ipu=[40], ipus_per_replica=1, enable_half_partials=True,
                        executable_cache_dir=executable_cache_dir)
ner_pipeline = pipelines.pipeline("ner", ipu_config=inference_config, padding='max_length', max_length=256)

In [ ]:
prompt = """The simplest way to use a model on the IPU is to use the `pipeline` function.
It provides a set of models which have been validated to work on a given task. To get
started choose the task and call the `pipeline` function"""
out = ner_pipeline(prompt)
out

In [ ]:
examples = [
    "My name is Wolfgang and I live in Berlin, I work for Graphcore and I really like HuggingFace",
    prompt,
    "I'm from France and I live in the UK, John is happy there."
]

In [ ]:
import gradio as gr
gr.Interface.from_pipeline(ner_pipeline, examples=examples).launch()

In [ ]:
model = "vblagoje/bert-english-uncased-finetuned-pos"
ner_pipeline_2 = pipelines.pipeline(
    "ner", model=model, ipu_config=inference_config,
    padding='max_length', max_length=256
)
out = ner_pipeline_2(prompt)

In [ ]:
gr.Interface.from_pipeline(ner_pipeline_2, examples=examples).launch()

In [ ]:
model = "chambliss/distilbert-for-food-extraction"
ner_pipeline_3 = pipelines.pipeline(
    "ner", model=model, ipu_config=inference_config,
    padding='max_length', max_length=256
)
out = ner_pipeline_3(prompt)

In [ ]:
gr.Interface.from_pipeline(
    ner_pipeline_3, examples=["I went to the restaurant last night, the food was excellent. I hate roast carrots with a side of chips, what a meal!"] + examples,
    description="Try prompting me with some food-related text!"
).launch()


In [ ]:
def from_pipeline(pipeline, examples=None, description=""):
    return gr.Interface(
        fn=lambda x: dict(text=x, entities=pipeline(x)),
        inputs = [
            gr.Textbox(
            label="Initial text",
            lines=3,
            value="The quick brown fox jumped over the lazy dogs.",
            examples=examples,
            description=description
            ),
        ],
        outputs=gr.HighlightedText(
            label="Diff",
            combine_adjacent=True,
        )
    )
    

In [ ]:
from_pipeline(ner_pipeline_3, examples=["I went to the restaurant last night, the food was excellent. I hate roast carrots with a side of chips, what a meal!"] + examples,
    description="Try prompting me with some food-related text!").launch()